#  SBIRT GPRA Coding Data Joining
Joing the multiple CSVs from the GPRA coding project due to the amount of pyton scripts fragmenting the data frame

In [1]:
import pandas as pd
import datetime as dt
from datetime import datetime

In [2]:
#Importing data
GPRA1 = pd.read_csv(r"\\cdchomedir01\home1\tyogarro\GPRA\GPRA1_SBIRT.csv", parse_dates = ['InterviewDate'])
GPRA2 = pd.read_csv(r"\\cdchomedir01\home1\tyogarro\GPRA\GPRA2_SBIRT.csv")
GPRA3 = pd.read_csv(r"\\cdchomedir01\home1\tyogarro\GPRA\GPRA3_SBIRT.csv")
GPRA4 = pd.read_csv(r"\\cdchomedir01\home1\tyogarro\GPRA\GPRA4_SBIRT.csv")
GPRA5 = pd.read_csv(r"\\cdchomedir01\home1\tyogarro\GPRA\GPRA5_SBIRT.csv")
GPRA6 = pd.read_csv(r"\\cdchomedir01\home1\tyogarro\GPRA\GPRA6_SBIRT.csv")
sf_client_id = pd.read_excel(r"W:\SBIRT\NEW SBIRT Patients.xlsx", sheet_name = 'SF')
rt_client_id = pd.read_excel(r"W:\SBIRT\NEW SBIRT Patients.xlsx", sheet_name = 'RT')
bi_client_id = pd.read_excel(r"W:\SBIRT\NEW SBIRT Patients.xlsx", sheet_name = 'BI')
bt_client_id = pd.read_excel(r"W:\SBIRT\NEW SBIRT Patients.xlsx", sheet_name = 'BT')


In [3]:
# adding outcome column to ClientID datasets
sf_client_id['Outcome'] = 'SF' 
rt_client_id['Outcome'] = 'RT'
bi_client_id['Outcome'] = 'BI'
bt_client_id['Outcome'] = 'BT'

In [4]:
# creating list object to return relevant columns
client_id_columns = ['Outcome', 'First Name','Last Name','MRN', 'New SPARS ID']

#segmenting datasets
sf_client_id = sf_client_id[client_id_columns]
rt_client_id = rt_client_id[client_id_columns]
bi_client_id = bi_client_id[client_id_columns]
bt_client_id = bt_client_id[client_id_columns]

# concating datasets
client_id = pd.concat([sf_client_id,rt_client_id,bi_client_id,bt_client_id])

# returning rows without blank MRNs
client_id = client_id[~client_id['MRN'].isna()]

# reanaming 'New SPARS ID' column
client_id.rename(columns = {'New SPARS ID':'ClientID'}, inplace = True)

In [5]:
#Merging all of the GPRA files
Full_GPRA_File = GPRA1.merge(GPRA2, on = 'ClientID', how = 'left')\
            .merge(GPRA3, on = 'ClientID', how = 'left')\
            .merge(GPRA4, on = 'ClientID', how = 'left')\
            .merge(GPRA5, on = 'ClientID', how = 'left')\
            .merge(GPRA6, on = 'ClientID', how = 'left')

In [6]:
#Subsetting dataset for merge to create IntakeDate Column 
Intake_Date_Merge = Full_GPRA_File[['ClientID','InterviewType','InterviewDate']]

#Filtering dataset for intake
Intake_Date_Merge = Intake_Date_Merge[Intake_Date_Merge['InterviewType'] == 1]

#Subsetting dataset further
Intake_Date_Merge = Intake_Date_Merge[['ClientID', 'InterviewDate']]

#Renaming columns
Intake_Date_Merge.rename(columns = {'InterviewDate':'IntakeDate'}, inplace = True)

#Merging files
Full_GPRA_File_SBIRT = Full_GPRA_File.merge(Intake_Date_Merge, on= 'ClientID', how = 'left')

In [7]:
Full_GPRA_File_SBIRT = Full_GPRA_File.merge(client_id, on= 'MRN', how = 'left')

### Modify Current Upload Date Range

In [8]:
Current_Upload = '_04.07.25-04.13.25_'
## Next Upload: '_04.14.25-04.20.25_'

In [9]:
# Date Filter for current date range
Full_GPRA_File_SBIRT = Full_GPRA_File_SBIRT[Full_GPRA_File_SBIRT['InterviewDate']>='2025-04-07']

In [10]:
Full_GPRA_File_SBIRT['InterviewType'].unique()

array([1., 2., 5.])

In [11]:
# creating dataframes based on interview type
SBIRT_Intakes = Full_GPRA_File_SBIRT[Full_GPRA_File_SBIRT['InterviewType']==1]
SBIRT_Follow_Ups = Full_GPRA_File_SBIRT[Full_GPRA_File_SBIRT['InterviewType']== 2]
SBIRT_Discharges = Full_GPRA_File_SBIRT[Full_GPRA_File_SBIRT['InterviewType']== 5]

# loading dataframes to csv files in submission location
SBIRT_Intakes.to_csv('SBIRT_Batch_Upload'+Current_Upload+'Intakes.csv', index = False, header = True)
SBIRT_Follow_Ups.to_csv('SBIRT_Batch_Upload'+Current_Upload+'Follow-Ups.csv', index = False, header = True)
SBIRT_Discharges.to_csv('SBIRT_Batch_Upload'+Current_Upload+'Discharges.csv', index = False, header = True)